In [2]:
def select_function(ct_str, key):
    # DES SBox 1
    SBox1 = [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ]

    # Expansion permutation table (DES standard)
    E = [31, 0, 1, 2, 3, 4, 3, 4, 5, 6, 7, 8, 7, 8, 9, 10, 11, 12, 
        11, 12, 13, 14, 15, 16, 15, 16, 17, 18, 19, 20, 19, 20, 21, 
        22, 23, 24, 23, 24, 25, 26, 27, 28, 27, 28, 29, 30, 31, 0]

    # Convert hex ciphertext to binary
    ciphertext = int(ct_str, 16)
    L16 = (ciphertext >> 32) & 0xFFFFFFFF
    R16 = ciphertext & 0xFFFFFFFF

    # R15 is L16 in the DES Feistel structure
    R15 = L16

    # Apply Expansion (E) to R15
    def apply_expansion(r):
        expanded = 0
        for i, bit in enumerate(E):
            expanded |= ((r >> bit) & 1) << (47 - i)
        return expanded

    expanded_R15 = apply_expansion(R15)

    # XOR first 6 bits of expanded R15 with the key
    sbox_input = (expanded_R15 >> 42) & 0x3F  # First 6 bits of expanded_R15
    sbox_input ^= key

    # Compute SBox output
    row = ((sbox_input >> 5) & 1) * 2 + (sbox_input & 1)  # Row from outer bits
    col = (sbox_input >> 1) & 0xF  # Column from middle 4 bits
    sbox_output = SBox1[row][col]

    return (sbox_output >> 3) & 1

In [3]:
import pandas as pd
import numpy as np

# Load preprocessed data
trace_df = pd.read_csv("processed_trace_data.csv")
plaintexts = [p for p in trace_df['Plaintext']]
ciphertexts = [c for c in trace_df['Ciphertext']]
powers_at_t = trace_df['Power']

In [4]:

key_guesses = range(64)  # All possible 6-bit key guesses

selection_values = {}

for key_guess in key_guesses:
    print(key_guess)
    selection_values[key_guess] = []
    for c in ciphertexts:
        selection_value = select_function(c, key_guess)
        selection_values[key_guess].append(selection_value)

dom_values = []

for key_guess in key_guesses:
    sel_values = selection_values[key_guess]

    # Partition the power traces based on selection values
    group0_indices = [i for i, val in enumerate(sel_values) if val == 0]
    group1_indices = [i for i, val in enumerate(sel_values) if val == 1]
    
    # Extract power traces for each group
    group0_powers = [powers_at_t[i] for i in group0_indices]
    group1_powers = [powers_at_t[i] for i in group1_indices]
    
    # Compute average power consumption for each group at the time point of interest
    avg_group0 = np.mean(group0_powers)
    avg_group1 = np.mean(group1_powers)
    
    # Compute the difference of means (DOM)
    dom = avg_group1 - avg_group0 
    dom_values.append(dom)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [5]:
max_dom = max(dom_values, key=abs)
correct_key_guess = dom_values.index(max_dom)
print(f"The key guess with the highest DOM is {correct_key_guess} with a value of {max_dom}")

The key guess with the highest DOM is 59 with a value of 0.00010812708322718856


In [6]:
dom_values

[2.628572627477732e-07,
 2.0158488456947787e-05,
 3.370800843623123e-05,
 2.7603009075491403e-05,
 -2.430929136165716e-05,
 7.035374784865323e-05,
 7.007957140155174e-05,
 -1.3532406605246128e-05,
 2.9369141504836938e-05,
 -3.573411820503858e-05,
 -4.201665278440181e-05,
 -3.0700322653343104e-05,
 -2.4464678667346362e-05,
 -4.53966777908682e-05,
 -1.534711255286869e-05,
 -2.0035293831516254e-05,
 -8.439172032639886e-05,
 -7.623770750991854e-05,
 -3.8384415089479096e-05,
 -5.093546780594871e-05,
 -1.1668086148650114e-05,
 -7.086516334059434e-05,
 -7.540249411913669e-05,
 -3.664976384749885e-05,
 2.3719369257602096e-05,
 8.216891064825871e-05,
 7.669902318921396e-05,
 2.6540210219083704e-05,
 -1.8168310022548262e-06,
 0.00010565777657810249,
 8.396606855003153e-05,
 4.760557381200403e-05,
 8.724181755560834e-05,
 1.0654295386255952e-05,
 4.051129254997732e-05,
 4.410646637165172e-05,
 4.306394798757329e-05,
 -2.5037383971443505e-05,
 3.258527979796422e-05,
 6.822718944862749e-05,
 -8.747